In [1]:
import keras,os,matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
import numpy as np
import scipy

2022-11-24 20:55:09.516084: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-24 20:55:09.807463: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-24 20:55:09.878104: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-24 20:55:09.878120: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [ ]:
amd_images = len(os.listdir(r"/home/kmh/Desktop/newnewdataset/training/amd"))
dme_images = len(os.listdir(r"/home/kmh/Desktop/newnewdataset/training/dme"))
normal_images = len(os.listdir(r"/home/kmh/Desktop/newnewdataset/training/normal"))
data= {'amd': amd_images, 'dme': dme_images, 'normal': normal_images}
labels = list(data.keys()) 
count = list(data.values()) 

batchSize=16

plt.rcParams['figure.figsize'] = (8.0, 8.0)
plt.bar(labels, count, color=['tab:red', 'tab:green', 'tab:blue'])
plt.axis('on')
plt.xlabel("Labels") 
plt.ylabel("Count") 
plt.savefig('labels_vs_counts.png', transparent= False, bbox_inches= 'tight', dpi= 400)
plt.show() 

In [2]:
trdata = ImageDataGenerator(
    rescale= 1./255,
    zoom_range= (0.73, 0.9),
    horizontal_flip= True,
    rotation_range= 10,
    width_shift_range= 0.10,
    fill_mode= 'constant',
    height_shift_range= 0.10,   
    brightness_range= (0.55, 0.9),
)
traindata = trdata.flow_from_directory(
    directory="/home/kmh/Desktop/newnewdataset/training",
    target_size=(224,224),
    batch_size= 32,  
    class_mode= 'categorical',
    shuffle= True, 
    seed= 1337
)

tsdata = ImageDataGenerator(
    rescale= 1./255,
)
testdata = tsdata.flow_from_directory(
    directory="/home/kmh/Desktop/newnewdataset/testing", 
    target_size=(224,224),
    batch_size= 32,  
    class_mode= 'categorical',
    shuffle= True, 
    seed= 1337
)

Found 3141 images belonging to 3 classes.
Found 90 images belonging to 3 classes.


In [3]:
num_classes = len(traindata.class_indices)  
train_labels = traindata.classes 
train_labels = to_categorical(train_labels, num_classes=num_classes)
valid_labels = testdata.classes 
valid_labels = to_categorical(valid_labels, num_classes=num_classes)

nb_train_samples = len(traindata.filenames)  
nb_valid_samples = len(testdata.filenames)

In [4]:
model = Sequential()
model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=3, activation="softmax"))

model.summary()

2022-11-24 20:55:11.679181: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-24 20:55:11.679584: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-24 20:55:11.679601: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (kmh): /proc/driver/nvidia/version does not exist
2022-11-24 20:55:11.680323: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 128)     73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 112, 112, 128)     147584    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 128)      0         
 2D)                                                    

In [5]:
model.compile(optimizer= keras.optimizers.Adam(lr= 0.0001), loss= 'categorical_crossentropy', metrics= ['accuracy'])

/home/kmh/Desktop/SEM7/DL/proj-test/env/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [6]:
checkpoint = ModelCheckpoint(
   'baseline_model.h5',
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    mode='auto',
    save_weights_only=False,
    period=1
)

earlystop = EarlyStopping(
    monitor='val_loss',
    min_delta=0.001,
    patience=3,
    verbose=1,
    mode='auto'
)

# csvlogger = CSVLogger(
#     filename= "baseline_training_csv.log",
#     separator = ",",
#     append = False
# )

reduceLR = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=3,
    verbose=1, 
    mode='auto'
)

callbacks = [checkpoint, earlystop, reduceLR]

In [7]:
history = model.fit(
    traindata, 
    epochs = 30,
    steps_per_epoch = nb_train_samples//32,
    validation_data = testdata, 
    validation_steps = nb_valid_samples//32,
    verbose = 2,
    callbacks = callbacks,
    shuffle = True
)

Epoch 1/30

Epoch 1: val_loss improved from inf to 1.02142, saving model to baseline_model.h5
98/98 - 1112s - loss: 0.9002 - accuracy: 0.5458 - val_loss: 1.0214 - val_accuracy: 0.5312 - lr: 1.0000e-04 - 1112s/epoch - 11s/step
Epoch 2/30

Epoch 2: val_loss improved from 1.02142 to 0.74725, saving model to baseline_model.h5
98/98 - 1124s - loss: 0.6446 - accuracy: 0.6790 - val_loss: 0.7473 - val_accuracy: 0.6406 - lr: 1.0000e-04 - 1124s/epoch - 11s/step
Epoch 3/30

Epoch 3: val_loss did not improve from 0.74725
98/98 - 1137s - loss: 0.5450 - accuracy: 0.7202 - val_loss: 0.7974 - val_accuracy: 0.6719 - lr: 1.0000e-04 - 1137s/epoch - 12s/step
Epoch 4/30

Epoch 4: val_loss improved from 0.74725 to 0.68743, saving model to baseline_model.h5
98/98 - 1148s - loss: 0.5013 - accuracy: 0.7465 - val_loss: 0.6874 - val_accuracy: 0.5781 - lr: 1.0000e-04 - 1148s/epoch - 12s/step
Epoch 5/30

Epoch 5: val_loss did not improve from 0.68743
98/98 - 1113s - loss: 0.4380 - accuracy: 0.7861 - val_loss: 1.05